In [5]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)

file = 'sensitivity_10_1e5.csv'

def mad(df):
    return (df - df.median()).abs().median()

In [13]:
sens = pd.DataFrame(np.loadtxt(f'../out/{file}'), columns=['r1', 'r2', 'r3', 't1', 't2', 't3', 'det', 'det_true'])\
            .filter(['r1', 'r2', 'r3', 'det', 'det_true'])
sens = sens.groupby(['r1', 'r2', 'r3']).agg([mad, np.median]).reset_index()
sens['ratio'] = sens[('det', 'mad')] / sens[('det', 'median')]

sens

r1    r2    r3       det             det_true               ratio
                           mad     median       mad    median          
0    0.1   0.1   0.1  0.022266  14.312590  1.750557  2.518841  0.001556
1    0.1   0.1   1.0  0.002636  10.469039  1.760188  2.509755  0.000252
2    0.1   0.1  10.0  0.013415   5.379578  1.727879  2.494383  0.002494
3    0.1   1.0   0.1  0.001512   9.070700  1.810319  2.587702  0.000167
4    0.1   1.0   1.0  0.001117   7.873545  1.805144  2.590230  0.000142
5    0.1   1.0  10.0  0.010037   4.199752  1.729278  2.499147  0.002390
6    0.1  10.0   0.1  0.009294   2.570576  1.767242  2.553254  0.003616
7    0.1  10.0   1.0  0.002669   2.549140  1.777965  2.548916  0.001047
8    0.1  10.0  10.0  0.005267   2.173498  1.769052  2.533557  0.002423
9    1.0   0.1   0.1  0.000403   8.115368  1.729485  2.533428  0.000050
10   1.0   0.1   1.0  0.000444   6.679788  1.731993  2.532635  0.000066
11   1.0   0.1  10.0  0.008348   3.483420  1.683895  2.462499  0.002397
12   1.0   1.0   0.1  0.000135   6.329287  1.776020  2.581602  0.000021
13   1.0   1.0   1.0  0.000441   5.638844  1.780329  2.586560  0.000078
14   1.0   1.0  10.0  0.007310   3.053648  1.682354  2.453773  0.002394
15   1.0  10.0   0.1  0.002724   1.958054  1.743765  2.540660  0.001391
16   1.0  10.0   1.0  0.004856   1.954158  1.748123  2.542990  0.002485
17   1.0  10.0  10.0  0.001391   1.658463  1.734378  2.519733  0.000838
18  10.0   0.1   0.1  0.002908   1.017504  1.552176  2.326598  0.002858
19  10.0   0.1   1.0  0.003899   1.007661  1.333006  2.099643  0.003869
20  10.0   0.1  10.0  0.005130   0.789940  1.586282  2.370914  0.006494
21  10.0   1.0   0.1  0.004065   1.009530  1.616539  2.391447  0.004027
22  10.0   1.0   1.0  0.002845   1.001078  1.319504  2.063113  0.002842
23  10.0   1.0  10.0  0.005069   0.793223  1.567385  2.355212  0.006391
24  10.0  10.0   0.1  0.002610   0.737011  1.648769  2.434964  0.003541
25  10.0  10.0   1.0  0.003447   0.734944  1.644316  2.433620  0.004691
26  10.0  10.0  10.0  0.003100   0.644773  1.612025  2.403135  0.004808

In [20]:
import plotly.graph_objects as go
from plotly.colors import DEFAULT_PLOTLY_COLORS
from plotly.subplots import make_subplots
import re

fig = make_subplots(1, 3, subplot_titles=[
    f'$r_4 = {r3}$' for r3 in [0.1, 1, 10]
], horizontal_spacing=0.05, vertical_spacing=0.1)

for i, (r3, d1) in enumerate(sens.groupby('r3')):
    z = d1.sort_values(['r2', 'r1']).ratio.to_numpy().reshape(3, 3)
    z = np.flipud(z)
    fig.add_trace(go.Heatmap(
        z=z*100,
        x=['0.1', '1', '10'],
        y=['10', '1', '0.1'],
        coloraxis='coloraxis',
        text=z*100,
        texttemplate='%{text:.2f}%',
        textfont={'size': 8}
    ), row=1, col=i+1)

fig.update_layout(
    coloraxis={'colorscale': 'viridis'},
    height=300,
)
fig.update_xaxes(title='$r_2$', row=1)
fig.update_yaxes(title='$r_3$', col=1)
fig.show()
fig.write_image('../figures/sensitivity_mad_optim.png')